In [ ]:
import tensorflow as tf
import numpy as np
import keras
import numpy as np
from keras import models
from keras import layers
from keras.initializers import Constant
from keras import optimizers

sentences=[]
x=0
with open('canonical_smiles.txt', 'r') as smile:
    for line in smile:
        if x<=600000:
            sentences.append(line[:-1])
            x+=1
        else:
            break

word_sequence = list("".join(sentences))
word_list = list(set(list("".join(sentences))))
word_dict = {w: i for i, w in enumerate(word_list)}

skip_grams = []

for i in range(1, len(word_sequence) - 1):
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]
    for w in context:
        skip_grams.append([target, w])
        
def random_batch(data, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False)

    for i in random_index:
        random_inputs.append(data[i][0])  # target
        random_labels.append([data[i][1]])  # context word

    return random_inputs, random_labels

training_epoch = 300
learning_rate = 0.001
batch_size = 20
embedding_size = 128
num_sampled = 15
voc_size = len(word_list)

inputs = tf.placeholder(tf.int32, shape=[batch_size])
labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

embeddings = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
selected_embed = tf.nn.embedding_lookup(embeddings, inputs)

nce_weights = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
nce_biases = tf.Variable(tf.zeros([voc_size]))

loss = tf.reduce_mean(
            tf.nn.nce_loss(nce_weights, nce_biases, labels, selected_embed, num_sampled, voc_size))

train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for step in range(1, training_epoch + 1):
        batch_inputs, batch_labels = random_batch(skip_grams, batch_size)

        _, loss_val = sess.run([train_op, loss],
                               feed_dict={inputs: batch_inputs,
                                          labels: batch_labels})
        
        if step % 10 == 0:
            print("loss at step ", step, ": ", loss_val)
    
    trained_embeddings = embeddings.eval()
    
dix={}
with open("dic.txt", "r") as dic:
    for index, line in enumerate(dic.readlines()):
        dix[line[0]]=index
    
def smilecode(code):
    chlist=list(code)
    reslist=[]
    for item in chlist:
        reslist.append(dix[item])
    return reslist

temp_x=[]
temp_y=[]

with open("new 1.txt", "r") as toxic:
    for line in toxic:
        linlist=line.split("\t")
        temp_x.append(smilecode(linlist[0]))
        temp_y.append(int(linlist[5]))

from random import shuffle

ln = len(temp_x)
ind=[i for i in range(ln)]
shuffle(ind)

for i in ind:
    temp_x.append(temp_x[i])
    temp_y.append(temp_y[i])
temp_x=temp_x[ln:]
temp_y=temp_y[ln:]

n=int(ln*0.2)

train_data=temp_x[n:]
test_data=temp_x[:n]
train_labels=temp_y[n:]
test_labels=temp_y[:n]

def vectorize_sequences(sequences, dimension=56):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence]=1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Embedding
max_features = 51
maxlen = 56
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 128

for r in range(0.0005, 0.005, 0.0005):
    for dec in range(0.005, 0.05, 0.005):
        for i in range(5):
            model = models.Sequential()

            model.add(layers.Embedding(max_features,
                                       embedding_size,
                                       input_length=maxlen,
                                       embeddings_initializer=Constant(trained_embeddings),
                                       trainable=False))

            model.add(layers.Dropout(0.3))
            model.add(layers.Conv1D(filters,
                             kernel_size,
                             padding='valid',
                             activation='relu'))
            model.add(layers.MaxPooling1D(pool_size=pool_size))
            model.add(layers.Bidirectional(layers.GRU(lstm_output_size)))
            model.add(layers.Dense(1, activation='sigmoid'))

            model.compile(optimizer=optimizers.Adam(lr=r, decay=dec),
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

            history = model.fit(x_train,
                                y_train,
                                epochs=800,
                                validation_data=(x_test, y_test))

            history_dict = history.history
            acc = history_dict['acc']
            val_acc = history_dict['val_acc']
            
            accstr=" [ "
            for item in val_acc:
                accstr=accstr+item
            accstr=accstr+" ]\n"
            
            with open("acclog.txt", "a+") as acclog:
                acclog.write("lr="+r+" decay="+dec+" maxacc="+np.amax(val_acc)+accstr)